In [14]:
""" Initialize population

This function generates n random solutions (food sources) within the domain 
of the variables to form an initial population for the ABC algorithm

Input
- D: number of variables
- bounds_lower: lower bounds of variables in vector
- bounds_upper: upper bounds of variables in vector
- n: number of solutions in population


"""

function initialize_population(D, bounds_lower, bounds_upper, n)
    population = []   
    for i in 1:n
        food_source = collect(rand(bounds_lower[i]:bounds_upper[i]) for i in 1:D)
        append!(population, [food_source])
    end
        
    return population
end	




initialize_population (generic function with 1 method)

In [16]:
"""

Objective function for toy example y = x1^2 + x2^2 + x3^2 + x4^2

Calculation for 1 vector or n instances (vectors) in population

"""

function compute_objective(input, vector=true)
    if vector == true
        objective = sum(input[i]^2 for i in 1:length(input))
        output = objective
    else
        objectives_population = []
        
        for j in 1:size(input)[1]
            food_source = input[j, :]
            #objective = sum(food_source[i]^2 for i in 1:length(food_source))
            objective = sum(sum(food_source[i].^2 for i in 1:length(food_source)))
            append!(objectives_population, objective)
        end
        
        output = objectives_population
    end
    
    return output
end

compute_objective (generic function with 2 methods)

In [35]:
""" Fitness function

Input
- objective values

Output
- fitness values


"""

function compute_fitness(objective_values)
    fitness_values = []
    
    for i in 1:length(objective_values)
        objective_value = objective_values[i]
        
        if objective_value >= 0
            fitness = 1/(1+objective_value)
     
        else
            fitness = 1+abs(objective_value)
        end
        
        append!(fitness_values, fitness)
    end
    return fitness_values
end



compute_objective (generic function with 2 methods)

In [15]:
bounds_lower = [-5,-5,-5,-5]
bounds_upper = [5,5,5,5]
D=4
n=9
population = initialize_population(D, bounds_lower, bounds_upper, n)

9-element Array{Any,1}:
 [2, -4, -3, -3]
 [0, 5, -2, -2]
 [5, 2, 1, 3]
 [-5, 3, -2, 2]
 [-4, 0, 2, 3]
 [-5, 4, 5, -3]
 [-3, -1, 5, -3]
 [3, -5, -3, 1]
 [2, 2, 4, -2]

In [49]:
population

9-element Array{Any,1}:
 [2, -4, -3, -3]
 [0, 5, -2, -2]
 [5, 2, 1, 3]
 [-5, 3, -2, 2]
 [-4, 0, 2, 3]
 [-5, 4, 5, -3]
 [-3, -1, 5, -3]
 [3, -5, -3, 1]
 [2, 2, 4, -2]

In [50]:
[1 2 2 3; 2 2 3 4]

2×4 Array{Int64,2}:
 1  2  2  3
 2  2  3  4

In [37]:
objectives = compute_objective(population, false)

9-element Array{Any,1}:
 38
 33
 39
 42
 29
 75
 44
 44
 28

In [38]:
fitness = compute_fitness(objectives)

9-element Array{Any,1}:
 0.02564102564102564
 0.029411764705882353
 0.025
 0.023255813953488372
 0.03333333333333333
 0.013157894736842105
 0.022222222222222223
 0.022222222222222223
 0.034482758620689655

In [191]:
""" Scouting function

This function generates n random solutions (food sources) within the domain 
of the variables to form an initial population for the ABC algorithm

Input
- p: population matrix
- bounds_lower: lower bounds of variables in vector
- bounds_upper: upper bounds of variables in vector
- t: trials vector
- fitness
- objective
- l: limit


"""

function Scouting(p,bounds_lower,bounds_upper,trials,fitness,objective,limit, best_fitness,D)
        
        # check whether the trial vector exceed the limit value and importantly where
        index_exceed = trials .> limit
        if sum(index_exceed) >= 1 # there is minimal one case where we exceed the limit
            if sum(maximum(trials) .== trials) > 1 # multiple cases have the same maximum so chose randomly
                possible_scoutings = findall(trials .== maximum(trials))
                idx = rand(1:size(possible_scoutings)[1])
                scouting_array = possible_scoutings[idx]
            else # only one array has a maximum => chose this one 
            
                scouting_array = argmax(trials)
            end
        end
        
        pop = p[scouting_array]
        fit = fitness[scouting_array]
        obj = objective[scouting_array]
        trail = trials[scouting_array]
        
        if fit > best_fitness # if you find a new fitness that is better than your best_fitness before you store this new one
            best_fitness = fit 
            best_population = pop
            best_obj = obj
        end
        
        #creating random population
        sol_new = bounds_lower + (bounds_upper-bounds_lower) .* rand(D) # -5 *(10*rand)
        new_obj = compute_objective(sol_new,true)
        new_fit = compute_fitness(new_obj)
        
        # replacing the new population
        p[scouting_array] = sol_new
        fitness[scouting_array] = new_fit[1]
        objective[scouting_array] = new_obj
        trials[scouting_array] = 0
    
        return p, fitness, objective, trials, best_fitness, best_population, best_obj  


end



Scouting (generic function with 3 methods)

In [207]:
bounds_lower + (bounds_upper-bounds_lower) .* rand(D)

4-element Array{Float64,1}:
  2.4666215183816735
  0.24785851440356232
 -0.010395787577439641
 -0.30845110681619214

In [199]:
Np = 9
limit = 4
bounds_lower = [-5,-5,-5,-5]
bounds_upper = [5,5,5,5]
D=4
n=9
trials = zeros(Np, 1)
trials[2] = 5
trials[3] = 6
trials[4] = 6
trials

9×1 Array{Float64,2}:
 0.0
 5.0
 6.0
 6.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [211]:
population[argmax(trials)]

4-element Array{Float64,1}:
  1.63499052213602
 -2.624716660935502
 -0.20624440592476923
 -4.097224820618259

In [192]:
Scouting(population, bounds_lower,bounds_upper,trials,fitness,objectives,4, 0.025)

(Any[[2, -4, -3, -3], [-3.4055621946355186, -3.4866990776753704, 2.4261132044040075, -4.345318179469833], [1.63499052213602, -2.624716660935502, -0.20624440592476923, -4.097224820618259], [-0.12392322216934204, 3.248035816996552, -4.826479921323399, -0.23416313601768213], [-4, 0, 2, 3], [-5, 4, 5, -3], [-3, -1, 5, -3], [3, -5, -3, 1], [2, 2, 4, -2]], Any[0.02564102564102564, 0.020192743907895563, Any[0.036506850023710756], Any[0.02864112106147619], 0.03333333333333333, 0.013157894736842105, 0.022222222222222223, 0.022222222222222223, 0.034482758620689655], Any[38, 48.52273968120747, 26.392119543332612, 33.91483443869285, 29, 75, 44, 44, 28], [0.0; 0.0; … ; 0.0; 0.0])

In [193]:
population

9-element Array{Any,1}:
 [2, -4, -3, -3]
 [-3.4055621946355186, -3.4866990776753704, 2.4261132044040075, -4.345318179469833]
 [1.63499052213602, -2.624716660935502, -0.20624440592476923, -4.097224820618259]
 [-0.12392322216934204, 3.248035816996552, -4.826479921323399, -0.23416313601768213]
 [-4, 0, 2, 3]
 [-5, 4, 5, -3]
 [-3, -1, 5, -3]
 [3, -5, -3, 1]
 [2, 2, 4, -2]

In [194]:
fitness

9-element Array{Any,1}:
 0.02564102564102564
 0.020192743907895563
  Any[0.036506850023710756]
  Any[0.02864112106147619]
 0.03333333333333333
 0.013157894736842105
 0.022222222222222223
 0.022222222222222223
 0.034482758620689655

In [195]:
objectives

9-element Array{Any,1}:
 38
 48.52273968120747
 26.392119543332612
 33.91483443869285
 29
 75
 44
 44
 28